In [ ]:
!unzip /content/glove.zip -d /content/drive/MyDrive/dataset/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import xticks
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import tensorflow as tf
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from keras.preprocessing.sequence import pad_sequences
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding,Activation,Dropout
from keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D,LSTM

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/dataset/cleaned_train.csv")
df.columns

In [ ]:
df.drop(['text','location'],axis=1,inplace=True)

In [ ]:
df.keyword.fillna('unknown',inplace=True)
df.isnull().sum()

In [ ]:
df1=pd.read_csv("/content/drive/MyDrive/dataset/cleaned_test.csv")
df1.columns

In [ ]:
df1.drop(['text','location'],axis=1,inplace=True)

In [ ]:
df1.keyword.fillna('unknown',inplace=True)
df1.isnull().sum()

creation of vocabulary

In [ ]:
# Bag of Words model
from keras.preprocessing.text import Tokenizer

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
# lets use only tweet text to build the model
X = df.text_cleaned
y = df.target

In [ ]:
# Test train split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# create and apply tokenizer on the training dataset
tokenizer = create_tokenizer(X_train)
X_train_set = tokenizer.texts_to_matrix(X_train, mode = 'freq')

In [ ]:
# define the model
def define_model(n_words):
    # define network
    model = Sequential()
    model.add(Dense(128, input_shape=(n_words,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics = [get_score])
    # summarize defined model
    model.summary()
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [ ]:
# function to calculate f1 score for each epoch
import keras.backend as K
def get_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
# create the model
n_words = X_train_set.shape[1]
model = define_model(n_words)

In [ ]:
#fit network
model.fit(X_train_set,y_train,epochs=10,verbose=2)

In [ ]:
# prediction on the test dataset
X_test_set = tokenizer.texts_to_matrix(X_test, mode = 'freq')
#y_pred = model.predict_classes(X_test_set)
y_pred1=model.predict(X_test_set) 
y_pred=np.argmax(y_pred1,axis=1)

In [ ]:
# important metrices
print(classification_report(y_test, y_pred))

In [ ]:
# apply tokenizer on the test dataset
test_set1 = tokenizer.texts_to_matrix(df1.text_cleaned, mode = 'freq')

In [ ]:
# make predictions on the test dataset
#y_test_pred = model.predict_classes(test_set)
y_test_pred2=model.predict(test_set1) 
y_test_pred3=np.argmax(y_test_pred2,axis=1)

In [ ]:
test_id=df1.id

In [ ]:
len(test_id)

In [ ]:
# lets prepare for the prediction submission
sub = pd.DataFrame()
sub['Id'] = test_id
sub['target'] = y_test_pred3
sub.to_csv('submission_1.csv',index=False)

GLOVE with KERAS Word Embeddings

In [ ]:
# Fitting a tokenizer on text will create a list of unique words with an integer assigned to it
token = Tokenizer()
token.fit_on_texts(X_train.tolist())

In [ ]:
# lets save the size of the vocab
vocab_size = len(token.word_index) + 1

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/MyDrive/dataset/glove.6B.100d.txt', mode='rt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
# we will now perform the encoding
encoded_docs = token.texts_to_sequences(X_train.tolist())

# embedding layer require all the encoded sequences to be of the same length, lets take max lenght as 100
# and apply padding on the sequences which are of lower size
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

In [ ]:
# create a weight matrix for words in training docs
mis_spelled = []
embedding_matrix = zeros((vocab_size, 100))
for word, i in token.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        mis_spelled.append(word)

In [ ]:
# lets check how many words are not spelled correctly 
len(mis_spelled)

In [ ]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[get_score])
# summarize the model
model.summary()
# fit the model
model.fit(padded_docs, y_train, epochs=50, verbose=0)

In [ ]:
loss, accuracy = model.evaluate(padded_docs, y_train, verbose=0)

In [ ]:
print(accuracy)

In [ ]:
encoded_docs = token.texts_to_sequences(X_test.tolist())
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
# prediction on the test dataset
predict_x=model.predict(padded_docs) 
y_pred=np.argmax(predict_x,axis=1)

In [ ]:
encoded_docs = token.texts_to_sequences(df1.text_cleaned.tolist())
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
predict_y = model.predict(padded_docs)
y_test_pred=np.argmax(predict_y,axis=1)

In [ ]:
# lets prepare for the prediction submission
sub = pd.DataFrame()
sub['Id'] = test_id
sub['target'] = y_test_pred
sub.to_csv('submission_2.csv',index=False)

CNN with word Embeddings

In [ ]:
max_length = max([len(s) for s in df.text_cleaned])
print('Maximum length: %d' % max_length)

In [ ]:
# we will now perform the encoding
encoded_docs = token.texts_to_sequences(X_train.tolist())

# embedding layer require all the encoded sequences to be of the same length, lets take max lenght as 100
# and apply padding on the sequences which are of lower size

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

In [ ]:
# define the model
def define_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    model.summary()
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [ ]:
# define model
model = define_model(vocab_size, max_length)
# fit network
model.fit(padded_docs, y_train, epochs=10, verbose=2)

In [ ]:
loss, accuracy = model.evaluate(padded_docs, y_train, verbose=0)

In [ ]:
print(accuracy)


In [ ]:
encoded_docs = token.texts_to_sequences(X_test.tolist())
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# prediction on the test dataset

In [ ]:
preds = model.predict(padded_docs)
y_pred=np.argmax(preds,axis=1)

In [ ]:

encoded_docs = token.texts_to_sequences(df1.text_cleaned.tolist())
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
y_test_pred1 = model.predict(padded_docs)
y_test_pred=np.argmax(y_test_pred1,axis=1)

In [ ]:
# lets prepare for the prediction submission
sub = pd.DataFrame()
sub['Id'] = test_id
sub['target'] = y_test_pred
sub.to_csv('submission_cnn.csv',index=False)